## Optimização de hyper-parâmetros

A ideia é que esta função selecione um número X de diferentes redes.

Depois vamos criar uma função que para cada rede, vai fazer o treino e testar o score com os dados de validação. Essa função depois ordena consoante o melhor valor de AUC.

In [1]:
from sklearn.model_selection import ParameterSampler
import random as r
def selecaoHyperParametros(d,neuronios):
    lista_parametros = list(ParameterSampler(d, n_iter=10, random_state=10))
    r.seed(10)
    for var in lista_parametros:
        var['topologia'] = r.choices(neuronios,k=var['nrCamadas'])
    return lista_parametros

## Aqui estão os parametros que vamos optimizar

### Faltam ainda ver o dropout, early stopping e regularização

### Optimizer temos de ver os parâmetros existentes também:
- rmsprop
    - rho: float >= 0.
    - epsilon: float >= 0. Fuzz factor. If None, defaults to K.epsilon().
    - decay: float >= 0. Learning rate decay over each update.
- SGD
    - momentum: float >= 0. Parameter that accelerates SGD in the relevant direction and dampens oscillations.
    - decay: float >= 0. Learning rate decay over each update.
    - nesterov: boolean. Whether to apply Nesterov momentum.
- Adam
    - beta_1: float, 0 < beta < 1. Generally close to 1.
    - beta_2: float, 0 < beta < 1. Generally close to 1.
    - epsilon: float >= 0. Fuzz factor. If None, defaults to K.epsilon().
    - decay: float >= 0. Learning rate decay over each update.
    - amsgrad: boolean. Whether to apply the AMSGrad variant of this algorithm from the paper "On the Convergence of Adam and Beyond"
    
##### Todos têm learning rate!
##### Existem ainda mais optimizers mas não sei se vale a pena ver todos!

In [2]:
dicionario = {
    'nrCamadas':[1,2,3,4,5,6,7,8],
    'ativacao':['relu','elu','exponential','tanh','sigmoid','linear'],
    'epochs':[4,8,10,20],
    'batch_size':[64,128,256,512]
}
neuronios = [2,3,4,5,8,10,16,32,64,128]

In [3]:
parametros = selecaoHyperParametros(dicionario,neuronios)
for var in parametros:
    print()
    print(var)


{'nrCamadas': 1, 'epochs': 20, 'batch_size': 128, 'ativacao': 'sigmoid', 'topologia': [10]}

{'nrCamadas': 5, 'epochs': 8, 'batch_size': 512, 'ativacao': 'sigmoid', 'topologia': [8, 10, 4, 64, 64]}

{'nrCamadas': 1, 'epochs': 8, 'batch_size': 256, 'ativacao': 'tanh', 'topologia': [16]}

{'nrCamadas': 6, 'epochs': 4, 'batch_size': 256, 'ativacao': 'elu', 'topologia': [3, 10, 5, 4, 128, 128]}

{'nrCamadas': 3, 'epochs': 8, 'batch_size': 256, 'ativacao': 'linear', 'topologia': [2, 64, 16]}

{'nrCamadas': 4, 'epochs': 20, 'batch_size': 64, 'ativacao': 'relu', 'topologia': [5, 4, 16, 8]}

{'nrCamadas': 6, 'epochs': 10, 'batch_size': 64, 'ativacao': 'exponential', 'topologia': [16, 16, 3, 32, 128, 128]}

{'nrCamadas': 1, 'epochs': 4, 'batch_size': 256, 'ativacao': 'relu', 'topologia': [16]}

{'nrCamadas': 1, 'epochs': 10, 'batch_size': 256, 'ativacao': 'linear', 'topologia': [2]}

{'nrCamadas': 4, 'epochs': 20, 'batch_size': 256, 'ativacao': 'tanh', 'topologia': [2, 3, 128, 5]}
